In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
mnist = input_data.read_data_sets('MNIST_DATA', one_hot=True)

Extracting MNIST_DATA/train-images-idx3-ubyte.gz
Extracting MNIST_DATA/train-labels-idx1-ubyte.gz
Extracting MNIST_DATA/t10k-images-idx3-ubyte.gz
Extracting MNIST_DATA/t10k-labels-idx1-ubyte.gz


In [3]:
class ConvModel(object):    
    def __init__(self, lr, batch_size, iter_num):
        self.lr = lr
        self.batch_size = batch_size
        self.iter_num = iter_num
        
        self.X_flat = tf.placeholder(tf.float32, [None, 784])
        self.X = tf.reshape(self.X_flat, [-1, 28, 28, 1]) # 本次要用卷积进行运算，所以使用2维矩阵。从这个角度讲，利用了更多的位置信息。
        self.y = tf.placeholder(tf.float32, [None, 10])
        self.dropRate = tf.placeholder(tf.float32)


        # 32个神经元
        # kernel_size: An integer or tuple/list of 2 integers, specifying the height and width of the 2D convolution window.
        # Can be a single integer to specify the same value for all spatial dimensions. 5x5的滑窗
        conv1 = tf.layers.conv2d(self.X, 32, 5, padding='same', activation=tf.nn.relu,
                                     kernel_initializer=tf.truncated_normal_initializer(stddev=0.1, seed=0),
                                     bias_initializer=tf.constant_initializer(0.1))        
        conv1 = tf.layers.max_pooling2d(conv1 , 2,2)        
        conv2 = tf.layers.conv2d(conv1, 64, 5, padding='same', activation=tf.nn.relu,
                                     kernel_initializer=tf.truncated_normal_initializer(stddev=0.1, seed=0),
                                     bias_initializer=tf.constant_initializer(0.1))
        pool1 = tf.layers.max_pooling2d(conv2, 2,2)               
        flatten = tf.reshape(pool1 , [-1, 7*7*64])
        dense1 = tf.layers.dense(flatten, 1024,  activation=tf.nn.relu, use_bias=True,
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=0.1, seed=0),
                                 bias_initializer=tf.constant_initializer(0.1))
        dense1_ = tf.nn.dropout(dense1, self.dropRate)
        dense2 = tf.layers.dense(dense1_, 10, activation=tf.nn.relu, use_bias=True,
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=0.1, seed=0),
                                 bias_initializer=tf.constant_initializer(0.1))

        self.loss = tf.losses.softmax_cross_entropy(onehot_labels=self.y, logits=dense2)
        self.train_step = tf.train.AdamOptimizer(1e-4).minimize(self.loss )
        
        # 用于模型训练
        self.correct_prediction = tf.equal(tf.argmax(self.y, axis=1), tf.argmax(dense2, axis=1))
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))
        # 用于保存训练好的模型
        self.saver = tf.train.Saver()
    def train(self):
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())  # 先初始化所有变量。
            for i in range(self.iter_num):
                batch_x, batch_y = mnist.train.next_batch(self.batch_size)   # 读取一批数据
                loss, _= sess.run([self.loss, self.train_step], 
                                  feed_dict={self.X_flat: batch_x, self.y: batch_y, self.dropRate: 0.5})   # 每调用一次sess.run，就像拧开水管一样，所有self.loss和self.train_step涉及到的运算都会被调用一次。
                if i%1000 == 0:                        
                    train_accuracy = sess.run(self.accuracy, feed_dict={self.X_flat: batch_x, self.y: batch_y, self.dropRate: 1.})  # 把训练集数据装填进去
                    test_x, test_y = mnist.test.next_batch(self.batch_size)
                    test_accuracy = sess.run(self.accuracy, feed_dict={self.X_flat: test_x, self.y: test_y, self.dropRate: 1.})   # 把测试集数据装填进去
                    print ('iter\t%i\tloss\t%f\ttrain_accuracy\t%f\ttest_accuracy\t%f' % (i,loss,train_accuracy,test_accuracy))
            self.saver.save(sess, 'model/mnistCNNModel') # 保存模型

    def test(self):
        with tf.Session() as sess:
            self.saver.restore(sess, 'model/mnistCNNModel')
            Accuracy = []
            for i in range(int(10000/self.batch_size)):
                test_x, test_y = mnist.test.next_batch(self.batch_size)
                test_accuracy = sess.run(self.accuracy, feed_dict={self.X_flat: test_x, self.y: test_y, self.dropRate: 1.})
                Accuracy.append(test_accuracy)
            print('==' * 15) 
            print( 'Test Accuracy: ', np.mean(np.array(Accuracy))   ) 


In [4]:
model = ConvModel(0.001, 64, 30000)   # 学习率为0.001，每批传入64张图，训练30000次
model.train()      # 训练模型

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

iter	0	loss	11.177574	train_accuracy	0.109375	test_accuracy	0.125000
iter	1000	loss	0.169426	train_accuracy	0.968750	test_accuracy	0.984375
iter	2000	loss	0.028347	train_accuracy	1.000000	test_accuracy	1.000000
iter	3000	loss	0.044962	train_accuracy	0.984375	test_accuracy	0.984375
iter	4000	loss	0.182806	train_accuracy	0.937500	test_accuracy	0.953125
iter	5000	loss	0.028693	train_accuracy	0.984375	test_accuracy	1.000000
iter	6000	loss	0.020976	train_accuracy	1.000000	test_accuracy	0.968750
iter	7000	loss	0.001591	train_accuracy	1.000000	test_accuracy	1.000000
iter	8000	loss	0.012811	train_accuracy	1.000000	test_accuracy	0.984375
iter	9000	loss	0.002953	train_accuracy	1.000000	test_accuracy	0.953125
iter	10000	loss	0.042865	train_accuracy	0.984375	test_accuracy	1.000000
iter	11000	loss	0.009167	train_

In [ ]:
model.test()

In [5]:
help(tf.layers.conv2d)

Help on function conv2d in module tensorflow.python.layers.convolutional:

conv2d(inputs, filters, kernel_size, strides=(1, 1), padding='valid', data_format='channels_last', dilation_rate=(1, 1), activation=None, use_bias=True, kernel_initializer=None, bias_initializer=<tensorflow.python.ops.init_ops.Zeros object at 0x7fd7281e82b0>, kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None, trainable=True, name=None, reuse=None)
    Functional interface for the 2D convolution layer.
    
    This layer creates a convolution kernel that is convolved
    (actually cross-correlated) with the layer input to produce a tensor of
    outputs. If `use_bias` is True (and a `bias_initializer` is provided),
    a bias vector is created and added to the outputs. Finally, if
    `activation` is not `None`, it is applied to the outputs as well.
    
    Arguments:
      inputs: Tensor input.
      filters: Integer, the dimensionality of t